In [ ]:
import itertools

In [ ]:
with open('data/schooling.csv', 'w') as out_file:
    for i in itertools.islice(open('data/schooling.txt'), 1, 5):
        print >>out_file, i.replace('"','').replace(", ", ",")

In [ ]:
import datetime

In [ ]:
datetime.datetime.strptime('2016-1-8', '%Y-%m-%d')